In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import os

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

In [3]:
auth_url= client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

In [4]:
code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)


In [5]:
client = stravalib.client.Client(access_token=access_token)
athlete = client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))


athlete name Brian, athlete id 3360678.


In [6]:
import datetime
lap_start_date = datetime.datetime(2017, 9, 28)
lap_start_date

datetime.datetime(2017, 9, 28, 0, 0)

In [7]:
lap_start_date - datetime.timedelta(days=1)

datetime.datetime(2017, 9, 27, 0, 0)

In [8]:
runs = []
for activity in client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        #print(activity.id, activity.name, activity.type, activity.start_date, activity.description)
        new_activity = client.get_activity(activity.id)
        #print(new_activity.description)
        try:
            scores, color = new_activity.description.split(' ')
            try:
                team_score, opponent_score = scores.split('-')
            except ValueError:
                team_score, opponent_score, color = None, None, None     
        except ValueError:
            scores = None
            team_score, opponent_score, color = None, None, None           
        
        print(new_activity.start_date, team_score, opponent_score, color)
        runs.append(activity)

2017-09-29 17:16:08+00:00 3 0 Colors
2017-10-11 17:17:28+00:00 1 2 colors
2017-10-13 17:11:22+00:00 1 3 white
2017-10-16 17:13:20+00:00 1 2 colors
2017-10-18 17:10:52+00:00 1 2 white
2017-10-20 17:16:14+00:00 2 0 white
2017-10-23 17:11:10+00:00 0 4 white
2017-10-25 17:13:00+00:00 1 2 colors
2017-10-27 17:57:16+00:00 None None None
2017-10-30 17:15:12+00:00 0 2 colors
2017-11-01 17:11:39+00:00 1 3 white
2017-11-03 17:07:24+00:00 1 3 colors
2017-11-06 18:09:48+00:00 2 1 white
2017-11-08 18:17:26+00:00 0 3 colors
2017-11-15 18:07:22+00:00 3 1 white
2017-11-17 18:16:55+00:00 2 0 colors
2017-11-20 18:12:23+00:00 2 1 white
2017-11-22 18:14:53+00:00 None None None
2017-11-27 18:12:54+00:00 1 2 white
2017-11-29 18:14:38+00:00 3 1 colors
2017-12-01 18:14:16+00:00 3 1 white
2017-12-06 18:04:49+00:00 None None None
2017-12-08 18:10:57+00:00 3 1 white
2017-12-11 18:12:01+00:00 4 0 white
2017-12-13 18:10:29+00:00 1 3 white
2017-12-15 18:16:30+00:00 3 0 white


In [9]:
activity.description

In [10]:
for run in runs:
    print(run)
    for lap in runs[1].laps:
        print(lap.elapsed_time)

<Activity id=1207538661 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1225759439 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1228558114 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1233412458 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1236492513 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1239212460 name='

In [11]:
lap

<ActivityLap id=3810821033 name='Lap 33' resource_state=2>

In [12]:

def xml_to_float(element):
    if element is not None:
        return float(element[0].firstChild.data)
    else:
        return np.nan
        
def xml_to_time(element):
    if element is not None:
        return datetime.datetime.strptime(element[0].firstChild.data, '%Y-%m-%dT%H:%M:%S.%fZ')
#         return dateutil.parser.parse(element[0].firstChild.data)
    else:
        return np.nan

In [13]:
event_lookup = [
    '',
    'team_point',
    'opponent_point',
    'my_point',
    'game',
    'game'
]

In [23]:
class Dummy(object):
    pass

self = Dummy()

# filename = 'activity_2088811850.tcx'   
filename = 'activity_2101180747.tcx'   
       
    

games = []


for filename in os.listdir():
    if '.tcx' in filename:
        print(filename)

        game_num = 0
    
        doc = minidom.parse(filename)
        doc.normalize()
        self.tcx = doc.documentElement   

        points = []

        lap_starts = []
        for lap in self.tcx.getElementsByTagName('Lap'):
            lap_starts.append(lap.attributes.items()[0][1])

        lap_starts = [datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.000Z') for t in lap_starts]
        lap_starts = lap_starts[1:] # Drop first lap (start of day)

        press_delta = datetime.timedelta(0,2)

        presses = 1

        events = []

        for n in range(len(lap_starts) - 1):


            time = lap_starts[n]

            dt = lap_starts[n+1] - lap_starts[n]

            # print(presses, time, dt, dt > press_delta)


            if dt > press_delta:
                events.append([presses, time])
                presses = 1

            else:        
                presses += 1

        events.append([presses, time])
        
        for e in events:
            print(e)


        base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0}
        game = base_game
        for event in events:
            game['end_time'] = event[1]

            event_type = event_lookup[event[0]]
            if event_type == 'game':
                games.append(game)                
                game_num += 1
                game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num}
                added = True

            elif event_type == 'my_point':
                game[event_type] += 1
                game['team_point'] += 1
                added = False

            else:
                game[event_type] += 1
                added = False

        if not added:
            games.append(game)



activity_2088811850.tcx
[4, datetime.datetime(2017, 10, 11, 17, 36, 39)]
[1, datetime.datetime(2017, 10, 11, 17, 39, 5)]
[1, datetime.datetime(2017, 10, 11, 17, 40, 43)]
[2, datetime.datetime(2017, 10, 11, 17, 44, 43)]
[2, datetime.datetime(2017, 10, 11, 17, 45, 14)]
[3, datetime.datetime(2017, 10, 11, 17, 47, 53)]
[2, datetime.datetime(2017, 10, 11, 17, 49, 20)]
[2, datetime.datetime(2017, 10, 11, 17, 50, 20)]
[2, datetime.datetime(2017, 10, 11, 17, 51, 45)]
[4, datetime.datetime(2017, 10, 11, 17, 51, 53)]
[2, datetime.datetime(2017, 10, 11, 17, 53, 44)]
[1, datetime.datetime(2017, 10, 11, 17, 57, 45)]
[1, datetime.datetime(2017, 10, 11, 17, 59, 16)]
[1, datetime.datetime(2017, 10, 11, 18, 0, 32)]
[4, datetime.datetime(2017, 10, 11, 18, 0, 46)]
activity_2101180747.tcx
[1, datetime.datetime(2017, 10, 13, 17, 16, 13)]
[2, datetime.datetime(2017, 10, 13, 17, 16, 24)]
[1, datetime.datetime(2017, 10, 13, 17, 19, 36)]
[2, datetime.datetime(2017, 10, 13, 17, 22, 19)]
[2, datetime.datetime(20

[1, datetime.datetime(2017, 10, 30, 17, 15, 47)]
[1, datetime.datetime(2017, 10, 30, 17, 17, 40)]
[2, datetime.datetime(2017, 10, 30, 17, 20, 26)]
[1, datetime.datetime(2017, 10, 30, 17, 21, 53)]
[2, datetime.datetime(2017, 10, 30, 17, 25, 18)]
[2, datetime.datetime(2017, 10, 30, 17, 27, 8)]
[1, datetime.datetime(2017, 10, 30, 17, 28, 47)]
[2, datetime.datetime(2017, 10, 30, 17, 31, 42)]
[2, datetime.datetime(2017, 10, 30, 17, 37, 7)]
[4, datetime.datetime(2017, 10, 30, 17, 37, 26)]
[2, datetime.datetime(2017, 10, 30, 17, 43, 28)]
[1, datetime.datetime(2017, 10, 30, 17, 50, 31)]
[2, datetime.datetime(2017, 10, 30, 17, 51, 58)]
[2, datetime.datetime(2017, 10, 30, 17, 55, 18)]
[2, datetime.datetime(2017, 10, 30, 17, 57, 54)]
[2, datetime.datetime(2017, 10, 30, 18, 2, 1)]
[4, datetime.datetime(2017, 10, 30, 18, 2, 29)]
activity_2259313466.tcx
[2, datetime.datetime(2017, 11, 1, 17, 12, 39)]
[1, datetime.datetime(2017, 11, 1, 17, 13, 53)]
[2, datetime.datetime(2017, 11, 1, 17, 17, 2)]
[2, d

[1, datetime.datetime(2017, 11, 20, 18, 14, 1)]
[1, datetime.datetime(2017, 11, 20, 18, 16, 58)]
[1, datetime.datetime(2017, 11, 20, 18, 19, 44)]
[2, datetime.datetime(2017, 11, 20, 18, 22, 55)]
[2, datetime.datetime(2017, 11, 20, 18, 24, 26)]
[1, datetime.datetime(2017, 11, 20, 18, 26, 11)]
[2, datetime.datetime(2017, 11, 20, 18, 27, 32)]
[1, datetime.datetime(2017, 11, 20, 18, 31, 39)]
[4, datetime.datetime(2017, 11, 20, 18, 31, 59)]
[1, datetime.datetime(2017, 11, 20, 18, 40, 18)]
[2, datetime.datetime(2017, 11, 20, 18, 42, 38)]
[1, datetime.datetime(2017, 11, 20, 18, 47, 29)]
[2, datetime.datetime(2017, 11, 20, 18, 49, 4)]
[1, datetime.datetime(2017, 11, 20, 18, 51, 5)]
[2, datetime.datetime(2017, 11, 20, 18, 53, 51)]
[1, datetime.datetime(2017, 11, 20, 19, 1, 15)]
[4, datetime.datetime(2017, 11, 20, 19, 1, 33)]
activity_2345377845.tcx
[1, datetime.datetime(2017, 11, 20, 19, 1, 33)]
activity_2353611914.tcx
[2, datetime.datetime(2017, 11, 27, 18, 14, 12)]
[1, datetime.datetime(2017,

[4, datetime.datetime(2017, 12, 13, 19, 7, 11)]
activity_2381352363.tcx
[1, datetime.datetime(2017, 12, 15, 18, 17, 52)]
[2, datetime.datetime(2017, 12, 15, 18, 19, 14)]
[2, datetime.datetime(2017, 12, 15, 18, 20, 40)]
[1, datetime.datetime(2017, 12, 15, 18, 22, 10)]
[2, datetime.datetime(2017, 12, 15, 18, 25, 36)]
[2, datetime.datetime(2017, 12, 15, 18, 27, 30)]
[3, datetime.datetime(2017, 12, 15, 18, 29, 49)]
[3, datetime.datetime(2017, 12, 15, 18, 32, 12)]
[3, datetime.datetime(2017, 12, 15, 18, 35, 29)]
[4, datetime.datetime(2017, 12, 15, 18, 35, 47)]
[3, datetime.datetime(2017, 12, 15, 18, 37, 51)]
[2, datetime.datetime(2017, 12, 15, 18, 39, 10)]
[1, datetime.datetime(2017, 12, 15, 18, 40, 20)]
[3, datetime.datetime(2017, 12, 15, 18, 46, 13)]
[3, datetime.datetime(2017, 12, 15, 18, 48, 41)]
[2, datetime.datetime(2017, 12, 15, 18, 49, 43)]
[3, datetime.datetime(2017, 12, 15, 18, 50, 42)]
[4, datetime.datetime(2017, 12, 15, 18, 51, 8)]
[2, datetime.datetime(2017, 12, 15, 18, 55, 39)

In [24]:
import pandas as pd

In [25]:
df = pd.DataFrame(games)

In [26]:
df['date'] = df.end_time.apply(lambda x: datetime.date(x.year, x.month, x.day))

In [27]:
df = df.set_index(['date', 'game_num'])
df

end_time  my_point  opponent_point  team_point
date       game_num                                                          
2017-10-11 0        2017-10-11 17:36:39         0               0           0
           1        2017-10-11 17:51:53         1               5           3
           2        2017-10-11 18:00:46         0               1           3
2017-10-13 0        2017-10-13 17:29:01         0               5           3
           1        2017-10-13 17:51:36         0               5           4
           2        2017-10-13 18:03:47         1               1           5
           3        2017-10-13 18:16:08         1               3           1
2017-10-16 0        2017-10-16 17:30:04         0               5           3
           1        2017-10-16 17:45:03         1               5           3
           2        2017-10-16 17:57:21         2               2           5
2017-10-18 0        2017-10-18 17:28:57         1               3           5
           1        2017-10-18 17:47:06         0               5           3
           2        2017-10-18 18:04:36         1               5           3
2017-10-20 0        2017-10-20 17:29:36         0               5           3
           1        2017-10-20 17:50:20         0               5           4
           2        2017-10-20 18:05:16         2               3           3
2017-10-23 0        2017-10-23 17:22:44         1               5           3
           1        2017-10-23 17:35:32         0               5           2
           2        2017-10-23 17:56:52         2               5           4
2017-10-25 0        2017-10-25 17:32:08         2               5           3
           1        2017-10-25 17:48:00         0               5           2
           2        2017-10-25 18:04:22         1               2           5
2017-10-30 0        2017-10-30 17:37:26         0               5           4
           1        2017-10-30 18:02:29         0               5           1
2017-11-01 0        2017-11-01 17:24:40         1               4           3
           1        2017-11-01 17:39:48         1               3           5
           2        2017-11-01 17:51:57         0               5           1
           3        2017-11-01 17:59:54         1               3           1
2017-11-03 0        2017-11-03 17:25:24         2               5           4
           1        2017-11-03 17:39:10         1               4           5
...                                 ...       ...             ...         ...
2017-11-27 2        2017-11-27 19:02:45         0               2           3
2017-11-29 0        2017-11-29 18:27:41         1               2           5
           1        2017-11-29 18:41:28         1               1           5
           2        2017-11-29 19:02:50         0               4           5
           3        2017-11-29 19:10:18         0               3           0
2017-12-01 0        2017-12-01 18:24:46         1               1           5
           1        2017-12-01 19:02:48         1               5           9
           2        2017-12-01 19:12:52         0               3           2
2017-12-06 0        2017-12-06 18:13:03         0               5           1
           1        2017-12-06 18:21:22         2               0           5
           2        2017-12-06 18:27:16         0               2           2
2017-12-08 0        2017-12-08 18:29:54         1               5           4
           1        2017-12-08 18:48:29         1               1           5
           2        2017-12-08 19:03:18         3               3           4
           3        2017-12-08 19:16:27         1               2           5
2017-12-11 0        2017-12-11 18:25:55         1               2           5
           1        2017-12-11 18:42:07         0               4           4
           2        2017-12-11 19:06:44         1               4           6
2017-12-13 0        2017-12-13 18:24:10         0    

In [19]:
pdf = df

In [20]:
pdf['my_team'] = None
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None

In [21]:
pdf.head()

end_time  my_point  opponent_point  team_point  \
date       game_num                                                             
2017-10-11 0        2017-10-11 17:36:39         0               0           0   
           1        2017-10-11 17:51:53         1               5           3   
           2        2017-10-11 18:00:46         0               1           3   
2017-10-13 0        2017-10-13 17:29:01         0               5           3   
           1        2017-10-13 17:51:36         0               5           4   

                    my_team white_wins color_wins white_point color_point  
date       game_num                                                        
2017-10-11 0           None       None       None        None        None  
           1           None       None       None        None        None  
           2           None       None       None        None        None  
2017-10-13 0           None       None       None        None        None  
           1           None       None       None        None        None

In [22]:
for date, row in pdf.unstack().iterrows():
    print(date)
    start_date = date - datetime.timedelta(days=1)
    end_date = date + datetime.timedelta(days=1)
    
    
    for activity in client.get_activities(after=start_date, before=end_date):
        if 'ltimate' in activity.name and activity.type == 'Run':
            #print(activity.id, activity.name, activity.type, activity.start_date, activity.description)
            new_activity = client.get_activity(activity.id)
            #print(new_activity.description)
            try:
                scores, team = new_activity.description.split(' ')
                team = team.lower()
                try:
                    team_score, opponent_score = scores.split('-')
                except ValueError:
                    team_score, opponent_score, team = None, None, None     
            except ValueError:
                scores = None
                team_score, opponent_score, team = None, None, None           
            
            pdf.loc[date, 'my_team'] = team

            if 'white' in team:
                pdf.loc[date, 'my_team'] = 'white'
                pdf.loc[date, 'white_wins'] = team_score                
                pdf.loc[date, 'color_wins'] = opponent_score   
                pdf.loc[date, 'white_point'] = pdf.loc[date, 'team_point']                
                pdf.loc[date, 'color_point'] = pdf.loc[date, 'opponent_point']                

            else:                
                pdf.loc[date, 'my_team'] = 'color'
                pdf.loc[date, 'color_wins'] = team_score                
                pdf.loc[date, 'white_wins'] = opponent_score   
                pdf.loc[date, 'color_point'] = pdf.loc[date, 'team_point']                
                pdf.loc[date, 'white_point'] = pdf.loc[date, 'opponent_point']       
            
            

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
pdf.loc[date, 'team_point']    

In [ ]:
pdf